## Imports + Logging Utilities (TOP OF NOTEBOOK)

In [0]:
from pyspark.sql.functions import (
    col, max as spark_max, row_number, round
)
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from datetime import timedelta, datetime

## Logging function

In [0]:
from pyspark.sql.types import (
    StructType, StructField,
    StringType, TimestampType, LongType
)

log_schema = StructType([
    StructField("pipeline_name", StringType(), True),
    StructField("layer", StringType(), True),
    StructField("start_time", TimestampType(), True),
    StructField("end_time", TimestampType(), True),
    StructField("status", StringType(), True),
    StructField("records_processed", LongType(), True),
    StructField("records_rejected", LongType(), True),
    StructField("error_message", StringType(), True)
])


In [0]:
def log_pipeline_event(
    pipeline_name,
    layer,
    start_time,
    end_time,
    status,
    records_processed,
    records_rejected,
    error_message=None
):
    log_data = [(
        pipeline_name,
        layer,
        start_time,
        end_time,
        status,
        int(records_processed),
        int(records_rejected),
        error_message if error_message is not None else ""
    )]

    log_df = spark.createDataFrame(log_data, schema=log_schema)

    (
        log_df
        .write
        .format("delta")
        .mode("append")
        .saveAsTable("workspace.retail.pipeline_logs")
    )


### Pipeline Metadata

In [0]:
pipeline_name = "silver_sales_incremental"
layer = "silver"
start_time = datetime.now()

### TRY–EXCEPT WRAPPER

In [0]:
try:
    # read bronze delta
    bronze_path = "dbfs:/Volumes/workspace/retail/raw/bronze_delta/sales"
    bronze_df = spark.read.format("delta").load(bronze_path)

    print("Bronze count:", bronze_df.count())
    
    # incremental window logic
    last_ts = (
        spark.table("workspace.retail.silver_sales")
        .select(spark_max("transaction_timestamp"))
        .collect()[0][0]
    )

    if last_ts is None:
        incremental_df = bronze_df
    else:
        incremental_df = bronze_df.filter(
            col("transaction_timestamp") >= last_ts - timedelta(days=2)
        )

    print("Incremental count:", incremental_df.count())

    # Deduplication (Latest Ingestion Wins)
    w = Window.partitionBy("transaction_id") \
              .orderBy(col("ingestion_timestamp").desc())

    dedup_df = (
        incremental_df
        .withColumn("rn", row_number().over(w))
        .filter(col("rn") == 1)
        .drop("rn")
    )

    print("After dedup:", dedup_df.count())

    # FK + Basic Validation
    products_df = spark.table("workspace.retail.silver_products")
    stores_df = spark.table("workspace.retail.silver_stores")

    valid_df = (
        dedup_df
        .join(products_df.select("product_id"), "product_id", "left_semi")
        .join(stores_df.select("store_id"), "store_id", "left_semi")
        .filter(col("quantity") > 0)
    )

    invalid_df = (
        dedup_df
        .join(
            valid_df.select("transaction_id"),
            "transaction_id",
            "left_anti"
        )
    )

    # Calibration (ONLY on valid data)
    calibrated_df = valid_df.withColumn(
        "total_amount",
        round(col("quantity") * col("unit_price") - col("discount"), 2)
    )

    # TYPE NORMALIZATION
    def normalize_silver_types(df):
        return (
            df
            .withColumn("quantity", col("quantity").cast("int"))
            .withColumn("unit_price", col("unit_price").cast("double"))
            .withColumn("discount", col("discount").cast("double"))
            .withColumn("total_amount", col("total_amount").cast("double"))
        )

    calibrated_df = normalize_silver_types(calibrated_df)
    invalid_df = normalize_silver_types(invalid_df)

    # Align Columns EXACTLY to Silver Schema
    silver_cols = spark.table("workspace.retail.silver_sales").columns

    calibrated_aligned_df = calibrated_df.select(*silver_cols)
    invalid_aligned_df = invalid_df.select(*silver_cols)

    #MERGE into Silver Sales (Idempotent)
    silver_tbl = DeltaTable.forName(spark, "workspace.retail.silver_sales")

    (
        silver_tbl.alias("t")
        .merge(
            calibrated_aligned_df.alias("s"),
            "t.transaction_id = s.transaction_id"
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

    # Write Quarantine (Append)
    (
        invalid_aligned_df.write
        .format("delta")
        .mode("append")
        .saveAsTable("workspace.retail.silver_sales_quarantine")
    )

    # SUCCESS LOGGING (Phase 8)
    end_time = datetime.now()

    log_pipeline_event(
        pipeline_name,
        layer,
        start_time,
        end_time,
        status="SUCCESS",
        records_processed=calibrated_aligned_df.count(),
        records_rejected=invalid_aligned_df.count()
    )

except Exception as e:
    end_time = datetime.now()

    log_pipeline_event(
        pipeline_name,
        layer,
        start_time,
        end_time,
        status="FAILED",
        records_processed=0,
        records_rejected=0,
        error_message=str(e)
    )

    raise

Bronze count: 247
Incremental count: 247
After dedup: 196


### Show ALL Pipeline Logs (Main Evidence)

In [0]:
%sql
SELECT
    pipeline_name,
    layer,
    start_time,
    end_time,
    status,
    records_processed,
    records_rejected,
    error_message
FROM workspace.retail.pipeline_logs
ORDER BY start_time DESC;


pipeline_name layer start_time end_time status records_processed records_rejected error_message silver_sales_incremental silver 2025-12-31T13:58:53.308Z 2025-12-31T13:59:05.314Z SUCCESS 12 184 bronze_sales_ingestion bronze 2025-12-31T13:58:23.875Z 2025-12-31T13:58:26.795Z SUCCESS 247 0 silver_sales_incremental silver 2025-12-31T13:49:51.296Z 2025-12-31T13:49:54.074Z FAILED 0 0 [FAILED_READ_FILE.DBR_FILE_NOT_EXIST] Error while reading file dbfs:/Volumes/workspace/retail/raw/bronze_delta/sales/part-00000-f07d25ee-614a-41dd-8ca1-e7d262176d5e.c000.snappy.parquet. [DELTA_FILE_NOT_FOUND_DETAILED] File dbfs:/Volumes/workspace/retail/raw/bronze_delta/sales/part-00000-f07d25ee-614a-41dd-8ca1-e7d262176d5e.c000.snappy.parquet referenced in the transaction log cannot be found. This occurs when data has been manually deleted from the file system rather than using the table `DELETE` statement. For more information, see https://docs.databricks.com/delta/delta-intro.html#frequently-asked-questions
 SQLSTATE: KD001

JVM stacktrace:
org.apache.spark.SparkException
	at org.apache.spark.sql.errors.QueryExecutionErrors$.fileNotExistErrorDBR(QueryExecutionErrors.scala:1293)
	at org.apache.spark.sql.errors.QueryExecutionErrors.fileNotExistErrorDBR(QueryExecutionErrors.scala:-1)
	at com.databricks.photon.NativeIOBroker.unwrapExecutionException(NativeIOBroker.java:409)
	at com.databricks.photon.NativeIOBroker.recordMissingOrCorruptFile(NativeIOBroker.java:648)
	at 0xc331ca7 .RecordMissingOrCorruptFile(external/workspace_spark_4_0/photon/jni-wrappers/jni-io-broker.cc:128)
	at 0x7544d3f .TryAndMaybeSkipFileOnError(external/workspace_spark_4_0/photon/exec-nodes/file-scan-node.cc:290)
	at 0x7548b3f .ScheduleNFRFetches(external/workspace_spark_4_0/photon/exec-nodes/file-scan-node.cc:357)
	at 0x7548b3f .OpenFileBatch(external/workspace_spark_4_0/photon/exec-nodes/file-scan-node.cc:580)
	at 0x7546f87 .DoOpenImpl(external/workspace_spark_4_0/photon/exec-nodes/file-scan-node.cc:563)
	at 0x7546b23 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/file-scan-node.cc:468)
	at 0x74939f3 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/topk-node.cc:179)
	at 0x74939f3 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/window-node.cc:95)
	at 0x74939f3 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/filter-node.cc:138)
	at 0x74939f3 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/project-node.cc:75)
	at 0x74939f3 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/shuffle-sink-node.cc:182)
	at com.databricks.photon.JniApiImpl.open(JniApi.scala:-2)
	at com.databricks.photon.JniApi.open(JniApi.scala:-1)
	at com.databricks.photon.JniExecNode.open(JniExecNode.java:74)
	at com.databricks.photon.PhotonPreShuffleResultHandler.$anonfun$getResult$1(PhotonExec.scala:1306)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.scala:18)
	at com.databricks.photon.PhotonResultHandler.timeit(PhotonResultHandler.scala:30)
	at com.databricks.photon.PhotonResultHandler.timeit$(PhotonResultHandler.scala:28)
	at com.databricks.photon.PhotonPreShuffleResultHandler.timeit(PhotonExec.scala:1299)
	at com.databricks.photon.PhotonPreShuffleResultHandler.getResult(PhotonExec.scala:1306)
	at com.databricks.photon.PhotonBasicEvaluatorFactory$PhotonBasicEvaluator$$anon$1.open(PhotonBasicEvaluatorFactory.scala:258)
	at com.databricks.photon.PhotonBasicEvaluatorFactory$PhotonBasicEvaluator$$anon$1.hasNextImpl(PhotonBasicEvaluatorFactory.scala:263)
	at com.databricks.photon.PhotonBasicEvaluatorFactory$PhotonBasicEvaluator$$anon$1.$anonfun$hasNext$1(PhotonBasicEvaluatorFactory.scala:283)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.scala:17)
	at com.databricks.photon.metrics.BillableTimeTaskMetrics.withPhotonBilling(BillableTimeTaskMetrics.scala:71)
	at org.apache.spark.TaskContext.runFuncAsBillable(TaskContext.scala:273)
	at com.databricks.photon.PhotonBasicEvaluatorFactory$PhotonBasicEvaluator$$anon$1.hasNext(PhotonBasicEval

### Show Only Latest Run (Very Clean Output)

In [0]:
%sql
SELECT *
FROM workspace.retail.pipeline_logs
ORDER BY start_time DESC
LIMIT 1;


pipeline_name,layer,start_time,end_time,status,records_processed,records_rejected,error_message
silver_sales_incremental,silver,2025-12-31T13:58:53.308Z,2025-12-31T13:59:05.314Z,SUCCESS,12,184,


### Show Only Failed Runs

In [0]:
%sql
SELECT *
FROM workspace.retail.pipeline_logs
WHERE status = 'FAILED'
ORDER BY start_time DESC;


pipeline_name layer start_time end_time status records_processed records_rejected error_message silver_sales_incremental silver 2025-12-31T13:49:51.296Z 2025-12-31T13:49:54.074Z FAILED 0 0 [FAILED_READ_FILE.DBR_FILE_NOT_EXIST] Error while reading file dbfs:/Volumes/workspace/retail/raw/bronze_delta/sales/part-00000-f07d25ee-614a-41dd-8ca1-e7d262176d5e.c000.snappy.parquet. [DELTA_FILE_NOT_FOUND_DETAILED] File dbfs:/Volumes/workspace/retail/raw/bronze_delta/sales/part-00000-f07d25ee-614a-41dd-8ca1-e7d262176d5e.c000.snappy.parquet referenced in the transaction log cannot be found. This occurs when data has been manually deleted from the file system rather than using the table `DELETE` statement. For more information, see https://docs.databricks.com/delta/delta-intro.html#frequently-asked-questions
 SQLSTATE: KD001

JVM stacktrace:
org.apache.spark.SparkException
	at org.apache.spark.sql.errors.QueryExecutionErrors$.fileNotExistErrorDBR(QueryExecutionErrors.scala:1293)
	at org.apache.spark.sql.errors.QueryExecutionErrors.fileNotExistErrorDBR(QueryExecutionErrors.scala:-1)
	at com.databricks.photon.NativeIOBroker.unwrapExecutionException(NativeIOBroker.java:409)
	at com.databricks.photon.NativeIOBroker.recordMissingOrCorruptFile(NativeIOBroker.java:648)
	at 0xc331ca7 .RecordMissingOrCorruptFile(external/workspace_spark_4_0/photon/jni-wrappers/jni-io-broker.cc:128)
	at 0x7544d3f .TryAndMaybeSkipFileOnError(external/workspace_spark_4_0/photon/exec-nodes/file-scan-node.cc:290)
	at 0x7548b3f .ScheduleNFRFetches(external/workspace_spark_4_0/photon/exec-nodes/file-scan-node.cc:357)
	at 0x7548b3f .OpenFileBatch(external/workspace_spark_4_0/photon/exec-nodes/file-scan-node.cc:580)
	at 0x7546f87 .DoOpenImpl(external/workspace_spark_4_0/photon/exec-nodes/file-scan-node.cc:563)
	at 0x7546b23 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/file-scan-node.cc:468)
	at 0x74939f3 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/topk-node.cc:179)
	at 0x74939f3 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/window-node.cc:95)
	at 0x74939f3 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/filter-node.cc:138)
	at 0x74939f3 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/project-node.cc:75)
	at 0x74939f3 .OpenImpl(external/workspace_spark_4_0/photon/exec-nodes/shuffle-sink-node.cc:182)
	at com.databricks.photon.JniApiImpl.open(JniApi.scala:-2)
	at com.databricks.photon.JniApi.open(JniApi.scala:-1)
	at com.databricks.photon.JniExecNode.open(JniExecNode.java:74)
	at com.databricks.photon.PhotonPreShuffleResultHandler.$anonfun$getResult$1(PhotonExec.scala:1306)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.scala:18)
	at com.databricks.photon.PhotonResultHandler.timeit(PhotonResultHandler.scala:30)
	at com.databricks.photon.PhotonResultHandler.timeit$(PhotonResultHandler.scala:28)
	at com.databricks.photon.PhotonPreShuffleResultHandler.timeit(PhotonExec.scala:1299)
	at com.databricks.photon.PhotonPreShuffleResultHandler.getResult(PhotonExec.scala:1306)
	at com.databricks.photon.PhotonBasicEvaluatorFactory$PhotonBasicEvaluator$$anon$1.open(PhotonBasicEvaluatorFactory.scala:258)
	at com.databricks.photon.PhotonBasicEvaluatorFactory$PhotonBasicEvaluator$$anon$1.hasNextImpl(PhotonBasicEvaluatorFactory.scala:263)
	at com.databricks.photon.PhotonBasicEvaluatorFactory$PhotonBasicEvaluator$$anon$1.$anonfun$hasNext$1(PhotonBasicEvaluatorFactory.scala:283)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.scala:17)
	at com.databricks.photon.metrics.BillableTimeTaskMetrics.withPhotonBilling(BillableTimeTaskMetrics.scala:71)
	at org.apache.spark.TaskContext.runFuncAsBillable(TaskContext.scala:273)
	at com.databricks.photon.PhotonBasicEvaluatorFactory$PhotonBasicEvaluator$$anon$1.hasNext(PhotonBasicEvaluatorFactory.scala:283)
	at com.databricks.photon.CloseableIterator$$anon$10.hasNext(CloseableIterator.scala:211)
	at scala.collection.Iterator$$anon$9.hasNext(Iterator.scala:583)
	at scala.c

### Records Processed vs Rejected (Data Quality Proof)

In [0]:
%sql
SELECT
    pipeline_name,
    SUM(records_processed) AS total_processed,
    SUM(records_rejected) AS total_rejected
FROM workspace.retail.pipeline_logs
GROUP BY pipeline_name;


pipeline_name,total_processed,total_rejected
silver_sales_incremental,22,187
bronze_sales_ingestion,741,0
